In [21]:
from pathlib import Path
from tqdm.auto import tqdm

from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# ============== Models ============== 
from transformers import DetrImageProcessor, DetrForObjectDetection
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
import supervision as sv # tracker 
from ultralytics import YOLO # yolo model
# shut down loggings
import logging
logging.getLogger("ultralytics").setLevel(logging.ERROR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = torch.Generator(device=device).manual_seed(42)

device

device(type='cuda')

# DETR

In [15]:
detr_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
detr_model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm").to(device)

In [ ]:
image = get_frame("02", 1)
width, height = image.size

threshold = 0.2


inputs = detr_processor(images=image, return_tensors="pt")
outputs = detr_model(**inputs)
# post process boxes
results = detr_processor.post_process_object_detection(outputs, target_sizes=torch.tensor([[width, height]]), threshold=threshold)[0]

# Faster RCCN resnet 50

In [22]:
rccn_model  = fasterrcnn_resnet50_fpn(weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /home/aliaagheis/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [03:48<00:00, 732kB/s]  


# YOLO 11 n

In [16]:
yolo_model = YOLO("yolo11n.pt").to(device)

100%|██████████| 5.35M/5.35M [01:22<00:00, 67.8kB/s]


In [ ]:
Image.fromarray